## Assignment 7A

### Problem Details
Thermal diffusion in the Earth’s crust (FTCS).

#### Equation of Motion
$$\dfrac{\partial T}{\partial t}=D\dfrac{\partial^2 T}{\partial x^2}$$
where $D = 0.1\text{m}^2\ \text{day}^{-1}$ in this problem.

#### Boundary Conditions
$$\begin{align}
T_\text{surf}(t,0\text{m})&=A+B\sin\left(\dfrac{2\pi t}{\tau}\right)\\
T_\text{deep}(t,20\text{m})&=11^\circ C
\end{align}$$
where $\tau = 365$ days, $A = 10^\circ C$, and $B = 12^\circ C$.

#### Initial Condition
$$\begin{align}
T(x,0) &= T^0_\text{surf} + (T^0_\text{deep} − T^0_\text{surf})\dfrac{x}{20}
\end{align}$$

#### Finite Approximation of EoM
We can write the differentials as finite differences:
$$\begin{align}
\dfrac{\partial^2T}{\partial x^2}&\approx\dfrac{T(t,x+\Delta x)-2T(t,x)+T(t,x-\Delta x)}{(\Delta x)^2}\\
\dfrac{\partial T}{\partial t}&\approx\dfrac{T(t+\Delta t,x)-T(t,x)}{\Delta t}
\end{align}$$
And using the heat equation we see that:
$$\begin{align}
T(t+\Delta t,x)&\approx T(t,x)+\dfrac{D\Delta t}{(\Delta x)^2}\left[T(t,x+\Delta x)-2T(t,x)+T(t,x-\Delta x)\right]\\
\end{align}$$

#### Stability of Finite Approximation
From von Neumann analysis, we will set the constraint:
$$h\ll\dfrac{(\Delta x)^2}{2D}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Problem parameters
D = 0.1             # m^2/day
L,  Δx = 20.0, 0.1  # m
tf, Δt = 365, 1.0   # days

# Temperature functions
def T_deep(t_day: float) -> float:
    return 11  # °C
def T_surf(t_day: float) -> float:
    return 10 + 12 * np.sin(2 * np.pi * t_day / 365)  # °C
def T_initial(x: float) -> float:
    return T_surf(0) + (T_deep(0) - T_surf(0)) * (x / L)  # °C

# Solve the heat equation
def T_next(T: np.ndarray, t: float, Δx: float, Δt: float) -> np.ndarray:
    k = D * Δt / Δx**2
    T_next_array = np.zeros_like(T)
    T_next_array[0] = T_surf(t)
    T_next_array[-1] = T_deep(t)
    for i in range(1, len(T) - 1):
        T_next_array[i] = T[i] + k * (T[i + 1] - 2 * T[i] + T[i - 1])
    return T_next_array

# Plot the initial temperature profile
if False:
    x = np.linspace(0, L, 100)
    plt.plot(x, [T_initial(xi) for xi in x])
    plt.xlabel('x (m)')
    plt.ylabel('Temperature (°C)')
    plt.title('Initial temperature profile')
    plt.show()

# Plot the temperature profile after tf days
if True:
    x = np.arange(0, L + Δx, Δx)
    T = np.array([T_initial(xi) for xi in x])
    for t in range(int(tf // Δt)):
        T = T_next(T, t, Δx, Δt)
    plt.plot(x, T)
    plt.xlabel('x (m)')
    plt.ylabel('Temperature (°C)')
    plt.title('Temperature profile after {} days'.format(tf))
    plt.show()

